# Hand Written Digit Recognition

# Import libraries

In [5]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense,Flatten,Dropout
from keras.layers import Conv2D,MaxPooling2D
from keras import backend as K

#load data and split into train and test sets
(x_train,y_train),(x_test,y_test)=mnist.load_data()

x_train.shape , y_train.shape

((60000, 28, 28), (60000,))

# Reshaping the data

In [ ]:
# x_train=x_train.reshape(x_train.shape[0],28,28,1)
x_test=x_test.reshape(x_test.shape[0],28,28,1)

# convert class vectors to binary class matrices
y_train=keras.utils.to_categorical(y_train,10)
y_test=keras.utils.to_categorical(y_test,10)

x_train=x_train.astype('float32')
x_test=x_test.astype('float32')

#making all data to be in between o to 1
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# Model

In [7]:
input_shape=(28,28,1)
model=Sequential([ 
    # here 32 and 64 are no of filters and a 3x3 is the kernal-sixe that means size of filter 
    Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    Conv2D(64,(3,3),activation='relu'),
    #Maxpooling helps to reduce the the size
    MaxPooling2D(pool_size=(2,2)),
    # It randomnly drops out a fraction of neurons to avoid overfitting
    Dropout(0.25),
    # it flattens the dimension from 3D to 1D
    Flatten(),
    Dense(256,activation='relu'),
    Dropout(0.5),
    Dense(10,activation='softmax')
])
model.compile(loss=keras.losses.CategoricalCrossentropy(),optimizer=keras.optimizers.Adam(0.001),metrics=['accuracy'])

# Fit the Model

In [8]:
hist = model.fit(x_train, y_train,batch_size=128,epochs=10,verbose=1,validation_data=(x_test, y_test))
print("The model has successfully trained")

Epoch 1/10
469/469 [==============================] - 188s 393ms/step - loss: 0.1882 - accuracy: 0.9425 - val_loss: 0.0450 - val_accuracy: 0.9859
Epoch 2/10
469/469 [==============================] - 183s 390ms/step - loss: 0.0639 - accuracy: 0.9804 - val_loss: 0.0337 - val_accuracy: 0.9888
Epoch 3/10
469/469 [==============================] - 183s 390ms/step - loss: 0.0462 - accuracy: 0.9859 - val_loss: 0.0307 - val_accuracy: 0.9897
Epoch 4/10
469/469 [==============================] - 180s 384ms/step - loss: 0.0371 - accuracy: 0.9886 - val_loss: 0.0259 - val_accuracy: 0.9910
Epoch 5/10
469/469 [==============================] - 154s 329ms/step - loss: 0.0307 - accuracy: 0.9903 - val_loss: 0.0240 - val_accuracy: 0.9922
Epoch 6/10
469/469 [==============================] - 108s 229ms/step - loss: 0.0262 - accuracy: 0.9916 - val_loss: 0.0241 - val_accuracy: 0.9916
Epoch 7/10
469/469 [==============================] - 110s 235ms/step - loss: 0.0226 - accuracy: 0.9928 - val_loss: 0.0260 -

# Evaluation

In [9]:
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.03141001611948013
Test accuracy: 0.989799976348877


# Saving The Model

In [10]:
model.save('mnist.h5')
print("Saving the model as mnist.h5")

Saving the model as mnist.h5


# GUI to predict digits

In [11]:
from keras.models import load_model
from tkinter import *
import tkinter as tk
import win32gui
from PIL import ImageGrab, Image
import numpy as np

model = load_model('mnist.h5')

def predict_digit(img):
    #resize image to 28x28 pixels
    img = img.resize((28,28))
    #convert rgb to grayscale
    img = img.convert('L')
    img = np.array(img)
    #reshaping to support our model input and normalizing
    img = img.reshape(1,28,28,1)
    img = img/255.0
    #predicting the class
    res = model.predict([img])[0]
    return np.argmax(res), max(res)

class App(tk.Tk):
    def __init__(self):
        tk.Tk.__init__(self)

        self.x = self.y = 0

        # Creating elements
        self.canvas = tk.Canvas(self, width=300, height=300, bg = "white", cursor="cross")
        self.label = tk.Label(self, text="Thinking..", font=("Helvetica", 48))
        self.classify_btn = tk.Button(self, text = "Recognise", command =         self.classify_handwriting) 
        self.button_clear = tk.Button(self, text = "Clear", command = self.clear_all)

        # Grid structure
        self.canvas.grid(row=0, column=0, pady=2, sticky=W, )
        self.label.grid(row=0, column=1,pady=2, padx=2)
        self.classify_btn.grid(row=1, column=1, pady=2, padx=2)
        self.button_clear.grid(row=1, column=0, pady=2)

        #self.canvas.bind("<Motion>", self.start_pos)
        self.canvas.bind("<B1-Motion>", self.draw_lines)

    def clear_all(self):
        self.canvas.delete("all")

    def classify_handwriting(self):
        HWND = self.canvas.winfo_id() # get the handle of the canvas
        rect = win32gui.GetWindowRect(HWND) # get the coordinate of the canvas
        im = ImageGrab.grab(rect)

        digit, acc = predict_digit(im)
        self.label.configure(text= str(digit)+', '+ str(int(acc*100))+'%')

    def draw_lines(self, event):
        self.x = event.x
        self.y = event.y
        r=8
        self.canvas.create_oval(self.x-r, self.y-r, self.x + r, self.y + r, fill='black')

app = App()
mainloop()

1/1 [==============================] - 0s 30ms/step
